In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
with open("preprocessing.py") as f:
    exec(f.read())

2024-04-06 06:30:33.966595: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Extracting labels: 100%|██████████| 73734/73734 [22:28<00:00, 54.68it/s]  


saved
(73734, 2)
(256029, 2)


Extracting labels: 100%|██████████| 10000/10000 [01:12<00:00, 137.91it/s]


saved
(10000, 2)
(38285, 2)


Extracting labels: 100%|██████████| 13934/13934 [01:30<00:00, 154.72it/s]


saved
(13934, 2)
(53022, 2)


In [ ]:
import pandas as pd
import numpy as np
import transformers 

import params
from prepare_data import TokenLabelDataset, csv_preprocessing
from compute_metrics import compute_metrics

print(params.DEVICE)

model = transformers.AutoModelForTokenClassification.from_pretrained(params.MODEL_CKECKPOINT, num_labels=len(params.LABEL_LIST))
model.config.id2label = dict(enumerate(params.MODEL_CKECKPOINT))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}
model = model.to(params.DEVICE)

tokenizer = params.TOKENIZER

num_epochs = 0

args = transformers.TrainingArguments(
    evaluation_strategy = "epoch",
    output_dir=params.CHECKPOINTS_DIR,
    learning_rate = params.LR,
    per_device_train_batch_size = params.BATCH_SIZE,
    per_device_eval_batch_size = params.BATCH_SIZE,
    num_train_epochs = num_epochs, # epoch count !!!
    weight_decay = params.WEIGHT_DECAY,
)

data_collator = transformers.DataCollatorForTokenClassification(tokenizer)

trainer = transformers.Trainer(
    model,
    args,
    train_dataset = TokenLabelDataset(params.SAVE_DIR + 'validation.pkl'),
    eval_dataset = TokenLabelDataset(params.SAVE_DIR + 'validation.pkl'),
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

print(trainer.args.device)

In [ ]:
test_dataset = TokenLabelDataset(csv_preprocessing('../small_test.csv', '../small_test.pkl'))
predictions, labels, _ = trainer.predict(test_dataset)
predictions = np.argmax(predictions, axis=2)

true_predictions = [
    [params.LABEL_LIST[p] for (p, l) in zip(prediction, label)]
    for prediction, label in zip(predictions, labels)
]

In [5]:
test_df = pd.read_pickle('../small_test.pkl')

tokenized_words = [params.TOKENIZER.convert_ids_to_tokens(params.TOKENIZER(test_df.loc[i, "words"], 
                                                                           is_split_into_words=True,
                                                                           max_length=512, truncation=True,
                                                                           padding='max_length',)["input_ids"]) for i in range(len(test_df))]

In [6]:
word_ids = [params.TOKENIZER(test_df.loc[i, "words"], is_split_into_words=True, max_length=512, padding='max_length', truncation=True,).word_ids() 
          for i in range(len(test_df))]

In [7]:
for i in range(5):
    for j in range(len(test_df.loc[i, 'words'])):
        if test_df.loc[i, 'labels'][j] in params.LABEL_LIST[:-1]:
            print(test_df.loc[i, 'words'][j]+':' , test_df.loc[i, 'labels'][j])
    print()

Мировой: B-IND
судья: I-IND
правонарушение: B-CR
ответственность: B-CR
личность: B-IND


Мировой: B-IND
судья: I-IND
водитель: B-IND
АО: B-LE

АО: B-LE
освидетельствование: B-LAW

арест: B-LAW
правонарушение: B-CR



In [8]:
for i in range(5):
    # print('!!!!!!!', len(word_ids[i]), len(true_predictions[i], ))
    for j in range(len(true_predictions[i])):
        if true_predictions[i][j] in params.LABEL_LIST[:-1]:
            print(test_df.loc[i, 'words'][word_ids[i][j]] , '->' , tokenized_words[i][j]+':' , true_predictions[i][j])
    print()

Мировой -> Мир: B-IND
Мировой -> ##овой: B-IND
судья -> суд: I-IND
судья -> ##ья: I-IND
правонарушение -> право: B-CR
правонарушение -> ##нар: B-CR
правонарушение -> ##уш: B-CR
правонарушение -> ##ение: B-CR
ответственность -> ответ: B-CR
ответственность -> ##ственность: B-CR
личность -> лично: B-IND
личность -> ##сть: B-IND

Мировой -> Мир: B-IND
Мировой -> ##овой: B-IND
судья -> суд: I-IND
судья -> ##ья: I-IND

Мировой -> Мир: B-IND
Мировой -> ##овой: B-IND
судья -> суд: I-IND
судья -> ##ья: I-IND
водитель -> води: B-IND
водитель -> ##тель: B-IND

освидетельствование -> ##ство: B-LAW
АО -> АО: B-LE

арест -> ар: B-LAW
арест -> ##ест: B-LAW
арест -> ар: B-LAW
арест -> ##ест: B-LAW
правонарушение -> право: B-CR
правонарушение -> ##нар: B-CR
правонарушение -> ##уш: B-CR
правонарушение -> ##ение: B-CR

